<center><h1>Python for Data Analysis and Scientific Computing</h1></center>
<center><h2>Final Project</h2></center>
<center> By Sebastien Mascha and Thomas Le Couédic </center>
<center><i> s.mascha@berkeley.edu | tlc10@berkeley.edy </i></center>

In [1]:
__version__ = "1.0"
__author__ = "Sebastien Mascha, Thomas Le Couedic"
__email__ = "s.mascha@berkeley.edu, tlc10@berkeley.edu"

In [2]:
import statistics
import sys
print("Python and compiler version:")
print(sys.version)

# Provides n-dimensional array object
import numpy as np
# Data analysis and manipulation tool
import pandas as pd
# Fundamental package for scientific computing
import scipy

# Plotting library
%matplotlib inline
import matplotlib as mpl
# State-based interface to matplotlib
import matplotlib.pyplot as plt

# Data Mining and Machine Learning
# from sklearn import datasets
import cv2
print("\nOpenCV version: " + cv2.__version__)
import tensorflow as tf
print("TensorFlow version: " + tf.__version__)

Python and compiler version:
3.8.1 (default, Feb  2 2020, 08:49:34) 
[GCC 8.3.0]

OpenCV version: 4.5.1
TensorFlow version: 2.4.1


In [6]:
# Install local packages
# imagerie_pkg: image processing tool
! pip install /app/packages/steganography_tools/dist/steganography_tools-0.1.0-py3-none-any.whl
# steganography_tools: steganography tools

from steganography_tools import LSBSteganography
# >>> secret = lsb.hide("./tests/sample-files/Lenna.png", "Hello World")
# >>> secret.save("./Lenna-secret.png")
# >>> clear_message = lsb.reveal("./Lenna-secret.png")

Processing /app/packages/steganography_tools/dist/steganography_tools-0.1.0-py3-none-any.whl
steganography-tools is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [8]:
secret = LSBSteganography.hide("photo.png", "Hello World")
secret.save("./photo-secret.png")

In [11]:
clear_message = LSBSteganography.reveal("./photo-secret.png")
print(clear_message)

Hello World


In [3]:
! pip install bitarray

     |████████████████████████████████| 69 kB 13.8 MB/s eta 0:00:01
  ERROR: Command errored out with exit status 1:
   command: /app/.venv/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-frb96zqt/bitarray_e7d24a9fcf0548dcb552458950711027/setup.py'"'"'; __file__='"'"'/tmp/pip-install-frb96zqt/bitarray_e7d24a9fcf0548dcb552458950711027/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/pip-wheel-w9lo_966
       cwd: /tmp/pip-install-frb96zqt/bitarray_e7d24a9fcf0548dcb552458950711027/
  Complete output (22 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.linux-x86_64-3.8
  creating build/lib.linux-x86_64-3.8/bitarray
  copying bitarray/test_bitarray.py -> build/lib.linux-x86_64-3.8/bitarray
  copying bitarray/__init__.py -> build/lib.linux-x86_64-3.8/bita

In [2]:
import numpy as np
import pandas as pd
# from skimage.transform import rescale
# import skimage
import bitarray
import math
import cv2
import random
from scipy import fftpack

class lsb:
	"""
	Steganographic tools based on least significant bit (lsb).
	"""

	def __messageToBits(message):
		tag = "{:<10}".format(str(len(message)*8))
		message = tag+message
		code = bitarray.bitarray()
		code.frombytes(message.encode('utf-8'))
		code = "".join(['1' if x == True else '0' for x in code.tolist()])
		return code

	def __hideMessage(img, message):
		shape = img.shape
		img = img.flatten()
		code = list(message)
		code_len = len(code)
		for i,x in enumerate(img):
			if i*2 <code_len:
				zbits = list('{0:08b}'.format(x))[:6]+code[i*2:i*2+2]
				img[i] = int("".join(zbits), 2)
			else:
				return img.reshape(shape)
		return img.reshape(shape)

	def encode(cover_image, secret_message):
		"""
		Stores a secret_message (string) into a cover_image (image) using lsb steganography.
		"""

		image_height  = cover_image.shape[0]
		image_width   = cover_image.shape[1]
		image_chanels = cover_image.shape[2]

		image_size_bytes = image_height * image_width * image_chanels
		image_size_bits  = image_size_bytes * 8
		image_size_kbytes = image_size_bytes / 1024

		hide_size_bits  = image_size_bytes * 2
		hide_size_kbytes = (hide_size_bits / 8) / 1024

		message_size_kbytes = len(secret_message)/8000

		if message_size_kbytes > hide_size_kbytes:
			raise Exception('Message too big!')
		
		secret_message_bits = lsb.__messageToBits(secret_message)

		image_with_message = lsb.__hideMessage(cover_image, secret_message_bits)

		return image_with_message

	def decode(image):
		"""
		Retrives a secret_message (string) from an image (image) using lsb steganography.
		"""

		bit_message = ""
		bit_count = 0
		bit_length = 200
		for i,x in enumerate(image):
			for j,y in enumerate(x):
				for k,z in enumerate(y):
					zbits = '{0:08b}'.format(z)
					bit_message += zbits[-2:]
					bit_count += 2
					if bit_count == 80:
						try:
							decoded_tag = bitarray.bitarray(bit_message).tobytes().decode('utf-8')
							bit_length = int(decoded_tag)+80
							bit_message = ""
						except:
							raise Exception("Can't decode secret on this image!")
					elif bit_count >= bit_length:
						return bitarray.bitarray(bit_message).tobytes().decode('utf-8')
                    
help(lsb)

ModuleNotFoundError: No module named 'bitarray'

In [18]:
image = plt.imread("photo.png")
lsb.encode(image, "hello")

NameError: name 'bitarray' is not defined